<a href="https://colab.research.google.com/github/KaruDevs/AN-LISIS-DE-DATOS-DE-MIGRACI-N-CON-NUMPY-Y-PANDAS/blob/main/ConsolidadoM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install pandas plotly seaborn matplotlib

In [12]:
pip install flask pandas plotly seaborn matplotlib

In [ ]:
# app.py

from flask import Flask, render_template, send_file
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os

app = Flask(__name__)

# Ruta base
@app.route('/')
def index():
    return '<h2>Dashboard de Migración</h2><ul>' \
           '<li><a href="/grafico">Gráfico de Migrantes por País</a></li>' \
           '<li><a href="/correlacion">Mapa de Calor de Correlaciones</a></li></ul>'

# Ruta para gráfico de barras interactivo
@app.route('/grafico')
def grafico():
    df = pd.read_csv('migración.csv')

    # Corrección de codificación
    df['Pais_Origen'] = df['Pais_Origen'].str.encode('latin1').str.decode('utf-8')
    df['Pais_Destino'] = df['Pais_Destino'].str.encode('latin1').str.decode('utf-8')
    df['Razon_Migracion'] = df['Razon_Migracion'].str.encode('latin1').str.decode('utf-8')

    # Agrupamiento
    origen_total = df.groupby('Pais_Origen')['Cantidad_Migrantes'].sum().reset_index()

    # Gráfico con Plotly
    fig = px.bar(
        origen_total,
        x='Pais_Origen',
        y='Cantidad_Migrantes',
        title='Migrantes por País de Origen',
        labels={'Cantidad_Migrantes': 'Cantidad de Migrantes'},
        color='Cantidad_Migrantes',
        color_continuous_scale='Viridis'
    )

    # Guardar como HTML temporal
    fig.write_html('static/grafico.html')
    return render_template('grafico.html')

# Ruta para mapa de calor de correlaciones
@app.route('/correlacion')
def correlacion():
    df = pd.read_csv('migración.csv')

    # Corrección de codificación
    df['Pais_Origen'] = df['Pais_Origen'].str.encode('latin1').str.decode('utf-8')
    df['Pais_Destino'] = df['Pais_Destino'].str.encode('latin1').str.decode('utf-8')
    df['Razon_Migracion'] = df['Razon_Migracion'].str.encode('latin1').str.decode('utf-8')

    # Variables numéricas
    variables = df[['Cantidad_Migrantes', 'PIB_Origen', 'PIB_Destino', 'IDH_Origen', 'IDH_Destino']]

    # Mapa de calor
    plt.figure(figsize=(10, 6))
    sns.heatmap(variables.corr(), annot=True, cmap='coolwarm')
    plt.title('Correlaciones entre Variables Económicas y Migración')
    plt.tight_layout()
    plt.savefig('static/correlacion.png')
    plt.close()

    return '<h2>Mapa de Calor de Correlaciones</h2><img src="/static/correlacion.png">'

if __name__ == '__main__':
    os.makedirs('static', exist_ok=True)
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


In [ ]:
# analisis_migracion.py

import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el archivo CSV
df = pd.read_csv('migración.csv')

# Corrección de caracteres mal codificados
df['Pais_Origen'] = df['Pais_Origen'].str.encode('latin1').str.decode('utf-8')
df['Pais_Destino'] = df['Pais_Destino'].str.encode('latin1').str.decode('utf-8')
df['Razon_Migracion'] = df['Razon_Migracion'].str.encode('latin1').str.decode('utf-8')

# Limpieza básica
df = df.drop_duplicates()
df = df.dropna(subset=['Pais_Origen', 'Pais_Destino', 'Cantidad_Migrantes'])
df['Cantidad_Migrantes'] = pd.to_numeric(df['Cantidad_Migrantes'], errors='coerce')

# Agrupamiento por país de origen
origen_total = df.groupby('Pais_Origen')['Cantidad_Migrantes'].sum().reset_index()

# Visualización interactiva con Plotly
fig_bar = px.bar(
    origen_total,
    x='Pais_Origen',
    y='Cantidad_Migrantes',
    title='Migrantes por País de Origen',
    labels={'Cantidad_Migrantes': 'Cantidad de Migrantes'},
    color='Cantidad_Migrantes',
    color_continuous_scale='Viridis'
)
fig_bar.show()

# Mapa de calor de correlaciones
variables = df[['Cantidad_Migrantes', 'PIB_Origen', 'PIB_Destino', 'IDH_Origen', 'IDH_Destino']]
plt.figure(figsize=(10, 6))
sns.heatmap(variables.corr(), annot=True, cmap='coolwarm')
plt.title('Correlaciones entre Variables Económicas y Migración')
plt.tight_layout()
plt.show()